In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math

# Part 1: Outcomes from No-regret Learning in Games

## Set Up and Algorithm Definition

In [2]:
class EWAlg:
    def __init__(self, epsilon, k, h, myBids, myValue):
        self.weights = np.ones(k)
        self.payoffs = np.zeros(k)
        self.h = h
        self.k = k
        self.epsilon = epsilon
        self.sumWeights = np.sum(self.weights)
        self.probs = self.weights/self.sumWeights
        self.myValue = myValue
        self.myBids = myBids

    def getBids(self):
        return self.myBids

    def getValue(self):
        return self.myValue

    def getAction(self):
        j = np.random.choice(self.k, 1, p = self.probs)
        return j.item()
    
    def update(self, payoffs):
        for j in range(len(payoffs)):
            curPayoff = payoffs[j]
            self.payoffs[j] = self.payoffs[j] + curPayoff
            newWeight = (1+self.epsilon)**(self.payoffs[j]/self.h)
            self.weights[j] = newWeight
        self.sumWeights = np.sum(self.weights)
        if(self.epsilon > 1):
            self.weights = self.weights/self.sumWeights
            self.sumWeights = np.sum(self.weights)
        self.probs = self.weights/self.sumWeights
        return


In [4]:
class FirstPriceReserve:
    # bidders have to be ordered the same way every time
    # reserve given as a 
    def __init__(self, bids, reserve):
        self.numBidders = bids.size
        self.totalPayoffs = np.zeros(self.numBidders)
        self.bids = bids
        self.reserve = reserve
        
    def generate(self):
        winningBid = 0
        winner = -1
        tiedBidders = []
        # check bids of all bidders
        for count, bid in enumerate(self.bids):
            if bid > winningBid:
                winningBid = bid
                winner = count
            elif (bid == winningBid) and (winner != -1):
                tiedBidders.add(count)
        winner = random.choice(tiedBidders)
        # see if winning bid is greater than reserve price
        if (winningBid < self.reserve): 
            winningBid = self.reserve
            winner = -1
        return winningBid, winner

In [ ]:
class mapBidToPayoffs:
    def __init__(self, myValue, possibleBids):
        self.myValue = myValue
        self.possibleBids = possibleBids
    
    def generatePayoffs(self, winningBid):
        payoffs = np.zeros(self.possibleBids.size)
        for count, bid in enumerate(self.possibleBids):
            if bid >= winningBid:
                payoffs[count] = bid - self.myValue
            else:
                payoffs[count] = 0
        return payoffs

In [ ]:
numPlayers = 2
epsilons = [2,5]
players = []
h = 1
k = 10
stepSize = 1/k * (np.log(h))
for count in range(numPlayers):
    # pick a distribution and numPlayers values from it
    playerValue = random.uniform(0,h)
    # create possible bids using geometric discretization
    playerBids = []
    for j in range(k):
        playerBids.append(playerValue - (1 + stepSize)**(j+1))
    # create player
    player = EWAlg(epsilons[count], k, h, playerBids, playerValue)
    players.append(player)

In [3]:
def MonteCarlo(numTrials, payoffGenerator, epsilon, k, h, n):
    avgFinalPayoff = 0
    avgRegretPerRound = [[] for i in range(numTrials)]
    for trial in range(numTrials):
        alg = EWAlg(epsilon, k, h)
        finalPayoff = 0
        actionPayoffs = np.zeros(k)
        generator = payoffGenerator(k)
        regretPerRound = np.zeros(n)
        for i in range(n):
            payoffs = generator.generate()
            j = alg.getAction()
            myPayoff = payoffs[j]
            actionPayoffs += payoffs
            alg.update(payoffs)
            finalPayoff += myPayoff
            OPT = max(actionPayoffs)
            regret = (OPT - finalPayoff).item() / (i+1)
            regretPerRound[i] = regret
        avgFinalPayoff += finalPayoff
        avgRegretPerRound[trial] = regretPerRound
    return avgFinalPayoff/numTrials, np.mean(avgRegretPerRound, axis=0)

In [3]:
def MonteCarloTrackActions(numTrials, payoffGenerator, epsilon, k, h, n):
    avgFinalPayoff = 0
    avgRegretPerRound = [[] for i in range(numTrials)]
    actionTrial = []
    for trial in range(numTrials):
        alg = EWAlg(epsilon, k, h)
        finalPayoff = 0
        actionPayoffs = np.zeros(k)
        generator = payoffGenerator(k)
        regretPerRound = np.zeros(n)
        actions = np.zeros(n)
        for i in range(n):
            payoffs = generator.generate()
            j = alg.getAction()
            actions[i] = j
            myPayoff = payoffs[j]
            actionPayoffs += payoffs
            alg.update(payoffs)
            finalPayoff += myPayoff
            OPT = max(actionPayoffs)
            regret = (OPT - finalPayoff).item() / (i+1)
            regretPerRound[i] = regret
        actionTrial.append(actions)
        avgFinalPayoff += finalPayoff
        avgRegretPerRound[trial] = regretPerRound
    return avgFinalPayoff/numTrials, np.mean(avgRegretPerRound, axis=0), np.array(actionTrial)

## Adversarial Fair Payoffs

In each round i:

Draw a payoff x ~ U[0,1] (i.e., from the uniform distribution on interval [0,1])

Assign this payoff to the action j* that has the smallest total payoff so far, i.e., j* = argminj Vji-1 where Vji = Σir=1 vji. 
(All other actions get 0 payoff in round i.)

In [202]:
h = 1 # fixed
# hyperparameters
k = 10
n = 100
epsilons = [-0.99, -0.5, 0, 0.1, 0.1517, 0.2, 0.4, 0.7, 1, 2, 10, 1000] # to be studied
monteCarloBound = 1000

AFEpsilonPayoffs = []
AFEpsilonRegretPerRound = []
for epsilon in epsilons:
    finalPayoff, regretPerRound = MonteCarlo(monteCarloBound, AdversarialFair, epsilon, k, h, n)
    AFEpsilonPayoffs.append(finalPayoff)
    AFEpsilonRegretPerRound.append(regretPerRound)

AFEpsilonAvgRegrets = [i[99] for i in AFEpsilonRegretPerRound]
print(epsilons)
print(AFEpsilonPayoffs)
print(AFEpsilonAvgRegrets)

[-0.99, -0.5, 0, 0.1, 0.1517, 0.2, 0.4, 0.7, 1, 2, 10, 1000]
[12.572066468624602, 6.089115420080595, 4.952127590364313, 4.888985005026019, 4.662385648248368, 4.658095671678926, 4.491083295527148, 4.237502645046786, 3.988782430758382, 3.551178754492665, 2.1475778776821812, 0.3206374990307352]
[-0.07164880805723829, -0.0066178097291542375, 0.0044772387027638095, 0.005445673155832275, 0.007507744684371933, 0.007499525986132154, 0.0091217183309684, 0.011695349045173858, 0.014306945820161048, 0.01850769358056152, 0.03260957846953832, 0.05091754046141372]
